**<font color = blue size=4>爬取網易雲動態歌詞(更新版)</font>**<br>

In [4]:
import os
import re
import csv
import ast 
import json
import time
import urllib
import random
import requests
import datetime
import numpy as np
import pandas as pd
import urllib.parse
from bs4 import BeautifulSoup
from user_agent import generate_user_agent

In [2]:
# 匯入歌手的csv檔案
def import_csv(filepath):
    df_artists = pd.read_csv(filepath, delimiter = ',')
    return df_artists

In [3]:
# 取得歌手對應的歌曲清單，收集歌單內的歌曲ID
# 取得歌曲頁面url，存成list(song_url)
# https://segmentfault.com/a/1190000014948845
def get_songs(artist_id):
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.117 Safari/537.36'}
    r = requests.get('http://music.163.com/artist?id={}'.format(artist_id),headers=headers)
    song_ids = re.findall(r'song\?id=(\d+?)".+?</a>',r.text) #歌曲id列表
    song_titles = re.findall(r'song\?id=\d+?">(.+?)</a>',r.text)#歌名title列表
    song_url = []
    for i in range(len(song_ids)):
        song_id = song_ids[i]
        url = 'https://music.163.com/#/song?id={}'.format(song_id)
        song_url.append(url)
    
    # 根據特定歌手ID搜尋歌手名稱
    # https://music.163.com/#/artist?id=1045123
    df_artists_filter = df_artists.loc[df_artists["artist_id"] == artist_id]
    artist_name = df_artists_filter['artist_name'].values[0]

    artist_url = 'http://music.163.com/artist?id=' + str(artist_id) #歌手頁面連結
    
    return [song_ids, song_titles, song_url, artist_name, artist_url]

In [4]:
# 歌單的歌轉成dataframe並存成csv
# 亂碼使用encoding='utf_8_sig'解決
def get_songinfo(song_ids, song_titles, song_url, artist_name):
    df_songs = pd.DataFrame()
    artist_name_list = []
    for i in range(len(song_ids)):
        artist_name_list.append(artist_name)
    df_songs['Artist'] = pd.Series(artist_name_list)
    df_songs['SongID'] = pd.Series(song_ids)
    df_songs['SongTitle'] = pd.Series(song_titles)
    df_songs['SongURL'] = pd.Series(song_url)
    artist_name_new = artist_name.replace(' / ',' ')
    filepath = 'CloudMusic_csv/artists/df_songs_' + artist_name_new + '.csv'
    df_songs.to_csv(filepath, encoding='utf_8_sig')
    return df_songs

In [5]:
# 檔案名稱所有非法字元都轉義：https://www.itread01.com/content/1549125182.html
def validateTitle(title):
    rstr = r"[\/\\\:\*\?\"\<\>\|]" # '/ \ : * ? " < > |'
    new_title = re.sub(rstr, "", title)  # 替換為空白
    return new_title

In [6]:
# 取得歌名和歌手
def get_name_artist(df_songs, count):
    artistname = df_songs['Artist'][count]
    artistname = validateTitle(artistname)
    songname = df_songs['SongTitle'][count]
    songname = validateTitle(songname)
    songid = df_songs['SongID'][count]
    songid = validateTitle(songid)
    #print(artistname + '_' + songname + '\n' + songid)
    return artistname, songname, songid

In [2]:
# 取得動態歌詞
# 取得動態歌詞
def get_lyrics(songid):
    headers={'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
             'Accept-Encoding': 'gzip, deflate',
             'Accept-Language': 'zh-CN,zh;q=0.9',
             'Connection': 'keep-alive',
             'Cookie': '_iuqxldmzr_=32; _ntes_nnid=0e6e1606eb78758c48c3fc823c6c57dd,1527314455632; '
                       '_ntes_nuid=0e6e1606eb78758c48c3fc823c6c57dd; __utmc=94650624; __utmz=94650624.1527314456.1.1.'
                       'utmcsr=(direct)|utmccn=(direct)|utmcmd=(none); WM_TID=blBrSVohtue8%2B6VgDkxOkJ2G0VyAgyOY;'
                       ' JSESSIONID-WYYY=Du06y%5Csx0ddxxx8n6G6Dwk97Dhy2vuMzYDhQY8D%2BmW3vlbshKsMRxS%2BJYEnvCCh%5CKY'
                       'x2hJ5xhmAy8W%5CT%2BKqwjWnTDaOzhlQj19AuJwMttOIh5T%5C05uByqO%2FWM%2F1ZS9sqjslE2AC8YD7h7Tt0Shufi'
                       '2d077U9tlBepCx048eEImRkXDkr%3A1527321477141; __utma=94650624.1687343966.1527314456.1527314456'
                       '.1527319890.2; __utmb=94650624.3.10.1527319890',
             'Host': 'music.163.com',
             'Referer': 'http://music.163.com/',
             'Upgrade-Insecure-Requests': '1',
             'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) '
                           'Chrome/66.0.3359.181 Safari/537.36'}
    url = 'http://music.163.com/api/song/lyric?id={}&lv=-1&kv=-1&tv=-1'.format(songid)
    user_agent = generate_user_agent()
    response = requests.get(url, headers={ 'user-agent': user_agent })
    # 隨機暫停 1~5 秒
    time.sleep(random.uniform(1,5))
    data = response.json()
    return data

In [8]:
# 下載動態歌詞，取代動態歌詞中的「’」符號，判斷回傳的lrc是否為空
# 解決UnicodeDecodeError: 'cp950'問題，加入encoding="utf-8"
def save_lrc_file(data, artistname, songname, count):
    content = data['lrc']['lyric']
    # lrc不為空
    if content != "":
        content = data['lrc']['lyric'].replace("’", "'")
        filepath = 'CloudMusic_lrc/artists/'+ artist_name + '_' + songname +'.lrc'
        #檔名重複篩選
        if os.path.isfile(filepath):
            #檔案存在
            pass
        else:
            #檔案不存在
            with open(filepath, 'w', encoding="utf-8") as file:
                file.write(content)
            print(filepath)
            count += 1
    # lrc為空
    else:
        pass
    return count

In [9]:
# 主程式抓歌曲
count_list = []
base = 0 #base,base+100
for w in range(0,1):
    print('Round:' + str(w))
    #artist_id = '45236'
    df_artists = import_csv('CloudMusic_csv/df_artists.csv')
    artist_id = df_artists['artist_id'][w]
    [song_ids, song_titles, song_url, artist_name, artist_url] = get_songs(artist_id)
    df_songs = get_songinfo(song_ids, song_titles, song_url, artist_name)
    count = 0
    for i in range(len(df_songs)):
        artistname, songname, songid = get_name_artist(df_songs, i)
        try:
            data = get_lyrics(songid)
            count = save_lrc_file(data, artistname, songname, count)    
        except:
            pass
    
    count_list.append(count)
    print('Artist: ' + artistname + ' / Count: ' + str(count))
    print('-'*80)

Round:0
Artist: Alan Walker / Count: 0
--------------------------------------------------------------------------------


In [5]:
get_lyrics(1296410418)

{'code': -460, 'msg': 'Cheating'}

In [15]:
# 有下載到的lrc數量
print(count_list)
print('LRC count:',len(count_list))

# 把每一位歌手抓到的lrc儲存起來並寫入csv
with open("CloudMusic_csv/df_songcount.csv", "a", encoding="utf-8", newline='') as csvfile:
    writer = csv.writer(csvfile)
    Round = str(base) + '-' + str(base+100)
    Count_list = count_list
    #第一次寫入
    #writer.writerow(['Round', 'Count_list'])
    #之後寫入
    writer.writerow([Round, Count_list])

df_songcount = pd.read_csv('CloudMusic_csv/df_songcount.csv', delimiter = ',')
df_songcount

In [60]:
# 儲存音樂
def save_mp3_file(filename, content):
    filepath = 'CloudMusic_mp3/'+ filename
    with open(filepath, mode = "wb") as f:
        f.write(content)
    print(filepath)

In [9]:
# 下載音樂.mp3
def download_music(songname, songid):
    # 取得Request URL
    url = 'http://music.163.com/song/media/outer/url?id=' + str(songid)+'.mp3'
    response = requests.get(url)
    content = response.content
    # mp3檔案命名規則
    save_mp3_file(songname + '_' + songid + '.mp3', content)

In [62]:
url = 'https://m801.music.126.net/20200113025447/8cd60d138777a13dd24f33d64be06b59/jdyyaac/520f/0158/0308/eefd2984f862ac2408b94bb0011ebcfa.m4a'
response = requests.get(url)
content = response.content
save_mp3_file('Alan Walker_Faded.mp3', content)

CloudMusic_mp3/Alan Walker_Faded.mp3
